In [ ]:
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import KNNWithMeans
from surprise.model_selection import KFold
from surprise import Reader
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.linalg import svds
import numpy as np

class KNNAlgo():
    def __init__(self,metric,n_neighbors):
        self.urls = None
        self.metric = metric
        self.n_neighbors = n_neighbors
    def set_values(self,urls,dataset):
        self.set_urls(urls)
        self.main_df = self.get_user_preferences_df()
        self.target_df = dataset
    def get_recom(self, df, x):
#         print(df.columns)
        reader = Reader(rating_scale=(1, 10))
        filterd_main = self.main_df[self.main_df['userID'].isin(self.target_df['userID'].to_list())]
        data = Dataset.load_from_df(df[['User-ID', 'ISBN', 'Book-Rating']], reader)
        books = df[['ISBN', 'Book-Title']].drop_duplicates(['ISBN', 'Book-Title'])
        users = df[['User-ID']].drop_duplicates(['User-ID'])
        print('printing this')
        sim_options = {'name': 'cosine', 'user_based': False}
        kf = KFold(n_splits=5)
        algo = KNNWithMeans(k=3, sim_options=sim_options)
        best_algo = None
        best_rmse = 1000.0
        best_pred = None
        rmse_list = []
        for trainset, testset in kf.split(data):
            # train and test algorithm.
            algo.fit(trainset)
            predictions = algo.test(testset)
            # Compute and print Root Mean Squared Error
            rmse = accuracy.rmse(predictions, verbose=True)
            rmse_list.append(rmse)
            if rmse < best_rmse:
                best_rmse = rmse
                best_algo = algo
                best_pred = predictions
        acc = [x * 19.6 for x in rmse_list]
        try:
            tsr_inner_id = best_algo.trainset.to_inner_uid(x)
            tsr_neighbors = best_algo.get_neighbors(tsr_inner_id, k=4)
            users = users[users['User-ID'].isin([algo.trainset.to_raw_uid(inner_id)
                          for inner_id in tsr_neighbors])]
        except ValueError:
            pass
        return users.head(10), acc

    def top_cosine_similarity(self, data, user_id, top_n=10):
        index = user_id
        book_row = data[index, :]
        magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
        similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
        sort_indexes = np.argsort(-similarity)
        return sort_indexes[:top_n]

    def similar_books(self, book_user_rating, x, top_indexes):
        print('Recommendations for {0}: \n'.format(x))
        recom_list = []
        for id in top_indexes + 1:
            recom_list.append(book_user_rating[book_user_rating['unique_user_id'] == id]['Book-Title'].unique())
        print((recom_list))

    def get_book_recom(self, ratings, users, books):
        user_rating_df = ratings.merge(users, left_on='User-ID', right_on='User-ID')
        book_user_rating = books.merge(user_rating_df, left_on='ISBN', right_on='ISBN')
        book_user_rating = book_user_rating.head(100000)
        book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
        book_user_rating.reset_index(drop=True, inplace=True)
        d = {}
        for i, j in enumerate(book_user_rating['User-ID'].unique()):
            d[j] = i
        book_user_rating['unique_user_id'] = book_user_rating['User-ID'].map(d)

        return book_user_rating

